In [29]:
import boto3
import pandas as pd
import numpy as np
import sys
from io import StringIO
import surprise
from surprise import accuracy
from sklearn.model_selection import train_test_split
from surprise import SVD, NMF, KNNBaseline
import pickle


In [30]:
sys.path.insert(1, '../src')
from Collaborative_recommender import CollaborativeRecommender
from Content_recommender import ContentRecommender
from Img_srcap import link
from NLP import nlp_process

In [31]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [32]:
%reload_ext autoreload

### Prepare lookup dataframes for the hybrid recommender 

In [33]:
s3 = boto3.client('s3')

In [13]:
df_all_15 = pd.read_csv("s3://recommender-system-amazon/df_all_15_text.csv", index_col=[0])

In [15]:
df_items = df_all_15.groupby(['asin', 'title','title_nlp','image_x'])['item_text'].apply(lambda x: ' '.join(x)).reset_index()

In [17]:
df_items = df_items[["asin","title","image_x"]]

In [20]:
df_items['links'] = df_items.apply(lambda x: link(x.image_x), axis=1)

In [26]:
df_items.drop('image_x',axis = 1, inplace = True)

In [69]:
df_items.set_index('asin', inplace = True)

In [64]:
df_items

,title,links
asin,,
7245456275,Black Mountain Products Single Resistance Band...,[https://images-na.ssl-images-amazon.com/image...
7245456313,Black Mountain Products Resistance Band Set wi...,[https://images-na.ssl-images-amazon.com/image...
B00003G4JR,Carrom Game Board Large,[https://images-na.ssl-images-amazon.com/image...
B00004NKIQ,Franklin Sports Adjustable Soccer Rebounder,[https://images-na.ssl-images-amazon.com/image...
B00004SYN6,Park &amp; Sun Sports Permanent Outdoor Tether...,[https://images-na.ssl-images-amazon.com/image...
...,...,...
B01HCU3NTU,CVLIFE Red &amp; Green Dot Sight 4 Reticles Re...,[https://images-na.ssl-images-amazon.com/image...
B01HD17S3A,"Ohuhu 28pcs Universal Hand Gun, Rifle &amp; Sh...",[https://images-na.ssl-images-amazon.com/image...
B01HE0PMFQ,ProFitness Weightlifting Belt (6-Inch-Wide) &n...,[https://images-na.ssl-images-amazon.com/image...


In [41]:
df_reviewer = df_all_15[['reviewerID',"asin"]]

In [43]:
df_reviewer

,reviewerID,asin
0,A1J5VYG8O0YEMB,7245456313
1,AYMDRRCOPKTCF,7245456313
2,A3AX86YSUHLELL,7245456313
3,A12MJJT1RL2S7K,7245456313
4,A33BKACRGR3H38,7245456313
...,...,...
228653,A1RL3SGTCTKOVA,B01HHB2HK0
228654,A1AZ1NXGGP6LDX,B01HHB2HK0
228655,A1S08PNPO3HXA5,B01HHB2HK0
228656,A2WVF9ZQ068DN0,B01HHB2HK0


Pickle dataframe for the flask app

In [65]:
with open('../../data/df_items.pkl', 'wb') as f:
    pickle.dump(df_items, f)

In [66]:
remote_file_name = 'df_items.pkl'
local_file_name = '../../data/df_items.pkl'
bucket_name = 'recommender-system-amazon'

s3.upload_file(Filename=local_file_name, 
               Bucket=bucket_name, 
               Key=remote_file_name)

In [44]:
with open('../../data/df_reviewer.pkl', 'wb') as f:
    pickle.dump(df_reviewer, f)

In [45]:
remote_file_name = 'df_reviewer.pkl'
local_file_name = '../../data/df_reviewer.pkl'
bucket_name = 'recommender-system-amazon'

s3.upload_file(Filename=local_file_name, 
               Bucket=bucket_name, 
               Key=remote_file_name)

## Loading the collaborative and content-based recommenders

Collaborative filter

In [48]:
with open('../../data/recom_colab.pkl', 'rb') as f:
    recom_colab = pickle.load(f)

Content based filter

In [50]:
with open('../../data/recom_content.pkl', 'rb') as f:
    recom_content = pickle.load(f)

Combining two recommenders

In [52]:
UI = "AZZG39P6RDCHL"
user_preference = recom_colab.get_top_n(UI,n=5)

In [53]:
user_preference

,predicted_rating
iid,
B000P3WQ86,5.0
B001ASUH4A,5.0
B004TBMUMM,5.0
B00AT6HZGM,5.0
B00GU3K5AQ,5.0


In [57]:
item = user_preference.index[0]

In [76]:
recommended_items = recom_content.get_recommendations(item)

In [77]:
df_items.loc[recommended_items]

,title,links
asin,,
B000P431BG,Plano Single Scoped or Double Non-Scoped Rifle...,[https://images-na.ssl-images-amazon.com/image...
B0029KH9FS,Plano 10470 Gun Guard SE Single Rifle Case,[https://images-na.ssl-images-amazon.com/image...
B00C3MFDKK,"Flambeau Outdoors 6500AR Tactical AR Case, Large",[https://images-na.ssl-images-amazon.com/image...
B001A76V2S,Bulldog Cases Pit Bull Rifle Case,[https://images-na.ssl-images-amazon.com/image...
B000P3WPHI,Plano Protector Single Pistol Case,[https://images-na.ssl-images-amazon.com/image...
